### Segmentação de usuários e análises

In [1]:
import pandas as pd
from scipy.stats import ttest_ind

df_base = pd.read_csv("/content/drive/MyDrive/Case Ifood/df_base_tratada.csv")
df_base.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 806467 entries, 0 to 806466
Data columns (total 7 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   customer_id    806466 non-null  object 
 1   grupo          806467 non-null  object 
 2   num_pedidos    806467 non-null  int64  
 3   gasto_total    806467 non-null  float64
 4   cliente_ativo  806467 non-null  bool   
 5   ticket_medio   806466 non-null  float64
 6   fez_pedido     806467 non-null  bool   
dtypes: bool(2), float64(2), int64(1), object(2)
memory usage: 32.3+ MB


#### Segmentações

In [2]:
# Segmentação por cliente ativo
df_base['segmento_atividade'] = df_base['cliente_ativo'].map({True: 'Ativo', False: 'Inativo'})

# Segmentação por frequência de pedidos
df_base['segmento_frequencia'] = df_base['num_pedidos'].apply(lambda x: '1 pedido' if x == 1 else 'Recorrente')

# Segmentação por ticket médio (quantis 0.33 e 0.66)
q1 = df_base['gasto_total'].quantile(0.33)
q2 = df_base['gasto_total'].quantile(0.66)

df_base['segmento_gasto_total'] = pd.cut(
    df_base['gasto_total'],
    bins=[-1, q1, q2, float('inf')],
    labels=[f'Baixo (≤ {q1:.2f})', f'Médio ({q1:.2f}–{q2:.2f})', f'Alto (> {q2:.2f})']
)


In [3]:
# Tabela-resumo por segmento
def resumo_segmento(df, coluna_segmento):
    resultados = []
    for segmento in df[coluna_segmento].dropna().unique():
        for grupo in ['target', 'control']:
            df_filt = df[(df[coluna_segmento] == segmento) & (df['grupo'] == grupo)]
            total_users = df_filt.shape[0]
            conversao = (df_filt['num_pedidos'] > 0).sum() / total_users if total_users > 0 else 0
            gasto_total = df_filt['gasto_total'].mean()
            num_pedidos = df_filt['num_pedidos'].mean()

            resultados.append({
                'segmento': segmento,
                'grupo': grupo,
                'usuarios': total_users,
                'conversao': round(conversao * 100, 2),
                'gasto_total_medio': round(gasto_total, 2),
                'num_pedidos_medio': round(num_pedidos, 2)
            })
    return pd.DataFrame(resultados)

# Testes estatísticos
def teste_segmento(df, coluna_segmento):
    for segmento in df[coluna_segmento].dropna().unique():
        df_seg = df[df[coluna_segmento] == segmento]

        gasto_target  = df_seg[df_seg['grupo'] == 'target']['gasto_total']
        gasto_control = df_seg[df_seg['grupo'] == 'control']['gasto_total']

        stat, pval = ttest_ind(gasto_target, gasto_control, equal_var=False)

        print(f"\n Segmento: {coluna_segmento} = {segmento}")
        print(f"- p-valor do teste t: {pval:.4f}")
        if pval < 0.05:
            print("- Diferença estatisticamente significativa")
        else:
            print("- Diferença **não** significativa")

# Executar para cada segmentação
for segmento in ['segmento_atividade', 'segmento_frequencia', 'segmento_gasto_total']:
    tabela = resumo_segmento(df_base, segmento)
    display(tabela)
    teste_segmento(df_base, segmento)

,segmento,grupo,usuarios,conversao,gasto_total_medio,num_pedidos_medio
0,Ativo,target,444861,100.00,228.63,4.79
1,Ativo,control,359700,100.00,202.60,4.23
2,Inativo,target,1064,99.91,282.50,5.85
3,Inativo,control,842,100.00,233.20,4.71



 Segmento: segmento_atividade = Ativo
- p-valor do teste t: 0.0000
- Diferença estatisticamente significativa

 Segmento: segmento_atividade = Inativo
- p-valor do teste t: 0.0078
- Diferença estatisticamente significativa


,segmento,grupo,usuarios,conversao,gasto_total_medio,num_pedidos_medio
0,Recorrente,target,354539,100.0,275.33,5.77
1,Recorrente,control,269341,100.0,255.02,5.33
2,1 pedido,target,91386,100.0,48.05,1.00
3,1 pedido,control,91201,100.0,48.08,1.00



 Segmento: segmento_frequencia = Recorrente
- p-valor do teste t: 0.0000
- Diferença estatisticamente significativa

 Segmento: segmento_frequencia = 1 pedido
- p-valor do teste t: 0.9151
- Diferença **não** significativa


,segmento,grupo,usuarios,conversao,gasto_total_medio,num_pedidos_medio
0,Alto (> 178.40),target,162430,100.0,486.93,9.25
1,Alto (> 178.40),control,111744,100.0,483.37,9.07
2,Baixo (≤ 70.70),target,136128,100.0,43.14,1.50
3,Baixo (≤ 70.70),control,130029,100.0,42.61,1.45
4,Médio (70.70–178.40),target,147367,100.0,115.65,2.91
5,Médio (70.70–178.40),control,118769,100.0,113.81,2.72



 Segmento: segmento_gasto_total = Alto (> 178.40)
- p-valor do teste t: 0.2485
- Diferença **não** significativa

 Segmento: segmento_gasto_total = Baixo (≤ 70.70)
- p-valor do teste t: 0.0000
- Diferença estatisticamente significativa

 Segmento: segmento_gasto_total = Médio (70.70–178.40)
- p-valor do teste t: 0.0000
- Diferença estatisticamente significativa


Baseado no gasto total, o cupom é mais efetivo para usuários ativos, frequentes e com gasto médio abaixo de 178 reais.

Usuários com gasto total alto ou que fizeram apenas um pedido não responderam significativamente ao cupom.

Isso sugere que a campanha pode ser mais eficiente e rentável se focada nesses segmentos, evitando custo em usuários menos responsivos.

#### Recomendações 1 - Variação de cenários e cupons

In [4]:
# Valores de cupom
valores_cupom = [5, 10, 15]

# Segmentos
cenarios_definidos = {
    "Todos os usuários (base)": (df_base['grupo'] == 'target'),
    "Ativos apenas": (df_base['grupo'] == 'target') & (df_base['segmento_atividade'] == 'Ativo'),
    "Ativos + Recorrentes": (
        (df_base['grupo'] == 'target') &
        (df_base['segmento_atividade'] == 'Ativo') &
        (df_base['segmento_frequencia'] == 'Recorrente')
    ),
    "Ativos + Recorrentes + Gasto < R$178": (
        (df_base['grupo'] == 'target') &
        (df_base['segmento_atividade'] == 'Ativo') &
        (df_base['segmento_frequencia'] == 'Recorrente') &
        (df_base['segmento_gasto_total'].isin(['Baixo (≤ 70.70)', 'Médio (70.70–178.40)']))
    ),

    "Gasto < R$178 apenas": (
        (df_base['grupo'] == 'target') &
        (df_base['segmento_gasto_total'].isin(['Baixo (≤ 70.70)', 'Médio (70.70–178.40)']))
    ),
    "Recorrentes + Alto Gasto (>178.40)": (
        (df_base['grupo'] == 'target') &
        (df_base['segmento_frequencia'] == 'Recorrente') &
        (df_base['segmento_gasto_total'] == 'Alto (> 178.40)')
    ),
    "1 pedido + Gasto Alto (> 178.40)": (
        (df_base['grupo'] == 'target') &
        (df_base['segmento_frequencia'] == '1 pedido') &
        (df_base['segmento_gasto_total'] == 'Alto (> 178.40)')
    ),
    "Ativos + Recorrentes + Médio (70.70–178.40)": (
    (df_base['grupo'] == 'target') &
    (df_base['segmento_atividade'] == 'Ativo') &
    (df_base['segmento_frequencia'] == 'Recorrente') &
    (df_base['segmento_gasto_total'] == 'Médio (70.70–178.40)')
    )
}

# Resultado
lista_resultados = []

# Loop para cada cenário e cada valor de cupom
for nome_cenario, filtro in cenarios_definidos.items():
    for valor_cupom in valores_cupom:
        df_target = df_base[filtro]
        df_control = df_base[
            (df_base['grupo'] == 'control') &
            (df_base['segmento_atividade'].isin(df_target['segmento_atividade'].unique())) &
            (df_base['segmento_frequencia'].isin(df_target['segmento_frequencia'].unique())) &
            (df_base['segmento_gasto_total'].isin(df_target['segmento_gasto_total'].unique()))
        ]

        num_usuarios = df_target['customer_id'].nunique()
        num_pedidos = df_target['num_pedidos'].sum()
        custo_total = num_usuarios * valor_cupom

        gasto_target = df_target['gasto_total'].mean()
        gasto_control = df_control['gasto_total'].mean()
        receita_inc_por_usuario = gasto_target - gasto_control
        num_usuarios = df_target['customer_id'].nunique()
        receita_inc_total = receita_inc_por_usuario * num_usuarios

        lucro = receita_inc_total - custo_total
        roi = (lucro / custo_total) * 100 if custo_total > 0 else None

        lista_resultados.append({
            'Cenário': nome_cenario,
            'Valor Cupom (R$)': valor_cupom,
            'Usuários': num_usuarios,
            'Pedidos': num_pedidos,
            'Custo Total (R$)': custo_total,
            'Receita Incremental (R$)': receita_inc_total,
            'Lucro Líquido (R$)': lucro,
            'ROI (%)': roi
        })

# Resultado final
df_resultados_cenarios = pd.DataFrame(lista_resultados)
df_resultados_cenarios.sort_values(['Cenário', 'Valor Cupom (R$)'], inplace=True)

df_resultados_cenarios['ROI (%)'] = df_resultados_cenarios['ROI (%)'].map('{:.2f}%'.format)

# Exibir
pd.options.display.float_format = 'R$ {:,.2f}'.format
display(df_resultados_cenarios)


,Cenário,Valor Cupom (R$),Usuários,Pedidos,Custo Total (R$),Receita Incremental (R$),Lucro Líquido (R$),ROI (%)
18,1 pedido + Gasto Alto (> 178.40),5,827,827,4135,"R$ 5,669.94","R$ 1,534.94",37.12%
19,1 pedido + Gasto Alto (> 178.40),10,827,827,8270,"R$ 5,669.94","R$ -2,600.06",-31.44%
20,1 pedido + Gasto Alto (> 178.40),15,827,827,12405,"R$ 5,669.94","R$ -6,735.06",-54.29%
6,Ativos + Recorrentes,5,353670,2039333,1768350,"R$ 7,165,854.45","R$ 5,397,504.45",305.23%
7,Ativos + Recorrentes,10,353670,2039333,3536700,"R$ 7,165,854.45","R$ 3,629,154.45",102.61%
8,Ativos + Recorrentes,15,353670,2039333,5305050,"R$ 7,165,854.45","R$ 1,860,804.45",35.08%
9,Ativos + Recorrentes + Gasto < R$178,5,192526,542079,962630,"R$ 603,500.26","R$ -359,129.74",-37.31%
10,Ativos + Recorrentes + Gasto < R$178,10,192526,542079,1925260,"R$ 603,500.26","R$ -1,321,759.74",-68.65%
11,Ativos + Recorrentes + Gasto < R$178,15,192526,542079,2887890,"R$ 603,500.26","R$ -2,284,389.74",-79.10%
21,Ativos + Recorrentes + Médio (70.70–178.40),5,134547,415734,672735,"R$ 230,456.08","R$ -442,278.92",-65.74%


#### Recomendações 2 - Cupom definido para cada segmentação

5 reais parece uma estratégia que funciona, mas ainda com o lucro baixo. Assim iremos testar uma nova estratégia:


In [5]:
def definir_valor_cupom(row):
    if row['segmento_gasto_total'] == 'Alto (> 178.40)' and row['segmento_frequencia'] == 'Recorrente':
        return 15
    elif row['segmento_gasto_total'] == 'Médio (70.70–178.40)' and row['segmento_frequencia'] == 'Recorrente':
        return 10
    elif row['segmento_frequencia'] == '1 pedido' or row['segmento_gasto_total'] == 'Baixo (≤ 70.70)':
        return 5
    else:
        return 0

# Aplicar apenas no grupo target
df_target = df_base[df_base['grupo'] == 'target'].copy()
df_target['valor_cupom'] = df_target.apply(definir_valor_cupom, axis=1)

# Controle com os mesmos segmentos
df_control = df_base[
    (df_base['grupo'] == 'control') &
    (df_base['segmento_gasto_total'].isin(df_target['segmento_gasto_total'].unique())) &
    (df_base['segmento_frequencia'].isin(df_target['segmento_frequencia'].unique()))
]

# KPIs
num_usuarios = df_target['customer_id'].nunique()
num_pedidos = df_target['num_pedidos'].sum()
custo_total = df_target['valor_cupom'].sum()
gasto_target = df_target['gasto_total'].mean()
gasto_control = df_control['gasto_total'].mean()
receita_incremental = (gasto_target - gasto_control) * num_usuarios
lucro = receita_incremental - custo_total
roi = (lucro / custo_total) * 100 if custo_total > 0 else None

# Resultado
print(f"Usuários: {num_usuarios}")
print(f"Pedidos: {num_pedidos}")
print(f"Custo total estimado da campanha: R$ {custo_total:,.2f}")
print(f"Receita incremental estimada: R$ {receita_incremental:,.2f}")
print(f"Lucro líquido estimado: R$ {lucro:,.2f}")
print(f"ROI da campanha: {roi:.2f}%")


Usuários: 445924
Pedidos: 2136745
Custo total estimado da campanha: R$ 4,519,810.00
Receita incremental estimada: R$ 11,630,902.72
Lucro líquido estimado: R$ 7,111,092.72
ROI da campanha: 157.33%


Ainda assim não funcionou bem. A ideia é ir testando até encontrar a melhor combinação de valor de cupom e segmentação